Please note that this tutorial is focused at case study.  If you are interested in more technical details, please consult the other tutorials.

# Requirements

## Environment

1. Anaconda Python distribution (tested with _Miniconda3-py39\_4.12.0-Linux-x86\_64.sh_, _conda v. 4.12.0_).
2. Jupyter server (see _extras/jupyter\_server.sh_ for details).
3. Anaconda environments (run _setup\_conda\_envs.sh_).
4. gmsh (not necessary if you already have meshes in either MSH or XDMF format).


## Setup

### Mesh for kESI

You need to have a mesh in XDMF format.  Run:

    conda activate kesi37
    cd extras/FEM/meshes
    snakemake meshes/four_spheres_csf_3_mm_plain/coarse.xdmf -j 1
    
It may take a while.


### Mesh for forward modelling

You will also need a mesh of slightly different geometry to simulate difference between assumed and actual brain.  Run:

    snakemake meshes/four_spheres_csf_1_mm_plain/normal.xdmf -j 1

#### Preprocessing

We are going to use:
  - the _extras/FEM/electrode\_locations/tutorial/case\_study.ini_ electrode positions,
  - the _extras/FEM/meshes/meshes/four\_spheres\_csf\_3\_mm\_plain/coarse.xdmf_ mesh, and
  - the _extras/FEM/model_properties/four\_spheres\_csf\_3\_mm.ini_ properties.

The preprocessing workflow is defined in the _extras/Snakefile_ and we run it with `snakemake`:

    cd extras
    snakemake .tutorial_case_study -j 1 --resources mem_mb=15000 --restart-times 2

That may take few **days**.  Ensure that you have enough diskspace (74+ GiB) at the filesystem of the _extras/FEM/solutions/tutorial/case\_study/_ directory.

> The `-j 1` parameter sets the number of jobs that can be run in parallel to 1.  If more CPU cores are available, you can increase that number to increase throughput, thus reduce the total preprocessing time.  But mind that in some systems (e.g. certain virtual machines) increase of paralellization beyond certain limit may significantly compromise performance.
> The `--resources mem_mb=15000` parameter tells the scheduler to possibly limit the total memory used by all simultaneous jobs to 15000 MiB.  You can adjust the parameter according to the value of the `-j` parameter and available free memory.  The scheduler limits the memory available for a job to 15000 MiB in case of calculating the leadfield correction, and to 7500 MiB in case of sampling the leadfield.  Both limits include about 2 GiB of safety margins.  Moreover, if a job fails, it is restarted (up to 2 times - given by the `--restart-times 2` parameter) with a doubled limit.

<!-- Solving requires at least 13.5 GB of memory, and sampling it - at least 5.7 GB -->

# Kernel construction

## Electrode object

An electrode object contains information about electrode spatial location (`.x`, `.y` and `.z` attribute), which is an absolute minimum to be used by kESI (in this case: kCSD with known base function in potential space).  It also provides additional information about:
- leadfield correction (`.correction_leadfield()` method) which enables kESI for setups violating kCSD assumptions,
  while facilitating application of analitically derived kCSD base functions to avoid significant numerical errors,
- base conductivity (`.base_conductivity` attribute) assumed when calculating the leadfield correction,
- grid used to sample the leadfield correction (`.SAMPLING_GRID` attribute).

In [ ]:
import numpy as np

class Electrode(object):
    def __init__(self, filename):
        """
        Parameters
        ----------
        
        filename : str
            Path to the sampled correction potential.
        """
        self.filename = filename
        with np.load(filename) as fh:
            self.SAMPLING_GRID = [fh[c] for c in 'XYZ']
            self.x, self.y, self.z = fh['LOCATION']
            self.base_conductivity = fh['BASE_CONDUCTIVITY']

    def correction_leadfield(self, X, Y, Z):
        """
        Correction of the leadfield of the electrode
        for violation of kCSD assumptions
        
        Parameters
        ----------
        X, Y, Z : np.array
            Coordinate matrices of the same shape.
        """
        with np.load(self.filename) as fh:
            return self._correction_leadfield(fh['CORRECTION_POTENTIAL'],
                                              [X, Y, Z])

    def _correction_leadfield(self, SAMPLES, XYZ):
        # if XYZ points are in nodes of the sampling grid,
        # no time-consuming interpolation is necessary
        return SAMPLES[self._sampling_grid_indices(XYZ)]

    def _sampling_grid_indices(self, XYZ):
        return tuple(np.searchsorted(GRID, COORD)
                     for GRID, COORD in zip(self.SAMPLING_GRID, XYZ))

We load electrode names as names of the sections of the config file.  Please note that the _fem_ section is not a name of an electrode.

In [ ]:
import glob

electrodes = [Electrode(filename)
              for filename in glob.glob('FEM/solutions/tutorial/case_study/sampled/9/*.npz')]

In [ ]:
len(electrodes)

## FRR: Fast Reciprocal Reconstructor 

The `_fast_reciprocal_reconstructor` experimental module contains tools which allow for fast construction of (cross)kernels.  They use discrete for high throughput integration.

### Convolver object

The convolver is the engine of the FRR tools.  It is used to:
- integrate leadfields weighted by a CSD profile,
- obtain CSD profile of a mixture of base functions.

The convolver operates on three regular 3D grids of coordinates:
- _POT_ grid used for leadfield (_reciprocal potential_) integration,
- _CSD_ grid used for CSD profile calculation,
- _SRC_ grid used for distributing of base function centroids.

The _SRC_ grid is an intersection (in set arithmetic sense) of the _POT_ and the _CSD_ grids, thus they define the convolver unequivocally.

In [ ]:
from _fast_reciprocal_reconstructor import ckESI_convolver

In [ ]:
_X, _Y, _Z = electrodes[0].SAMPLING_GRID

_pot_mesh = [_X, _Y, _Z]
_csd_mesh = [_X[::2], _Y[::2], _Z[::2]]

convolver = ckESI_convolver(_pot_mesh, _csd_mesh)

Open 3D meshgrids may be accessed as `.{NAME}_MESH` attributes, where `{NAME}` is the name of the mesh.
Components of each meshgrid may be accessed as `.{NAME}_{C}` attributes, where `{C}` is the name of the coordinate.

### Model source

While FRR tools operate on base function profiles defined as callables, we can use convenience kCSD base function objects as model bases (bases which centroid is `(0, 0, 0)`).
As convolver will use the Romberg method for integration, the size of the CSD profile is limited by the K parameter of the method.

In [ ]:
from _common_new import SphericalSplineSourceKCSD, GaussianSourceKCSD3D

ROMBERG_K = 6
SRC_R_MAX = 2 ** (ROMBERG_K - 1) * min(convolver.ds('POT'))
BASE_CONDUCTIVITY = electrodes[0].base_conductivity

spline_nodes = [SRC_R_MAX / 3, SRC_R_MAX]
spline_polynomials = [[1],
                      [0,
                       6.75 / SRC_R_MAX,
                       -13.5 / SRC_R_MAX ** 2,
                       6.75 / SRC_R_MAX ** 3]]
model_src = SphericalSplineSourceKCSD(0, 0, 0,
                                      spline_nodes,
                                      spline_polynomials,
                                      BASE_CONDUCTIVITY)
print(SRC_R_MAX)

### Convolver interface

The convolver interface binds the convolver to:
- a CSD profile,
- weights of a quadrature of equally-spaced nodes,
- boolean mask of nodes of the _SRC_ mesh with centroids of the base functions.

When analytical solution of the kCSD forward problem is used coupled with numeric integration of leadfield correction, it is advised not to put centroids near the boundary of the _SRC_ grid.

In [ ]:
from _fast_reciprocal_reconstructor import ConvolverInterfaceIndexed
from scipy.integrate import romb

ROMBERG_N = 2 ** ROMBERG_K + 1
ROMBERG_WEIGHTS = romb(np.identity(ROMBERG_N)) * 2 ** -ROMBERG_K
BRAIN_RADIUS = 0.079

SRC_IDX = ((convolver.SRC_Z > -0.06 + SRC_R_MAX)
           & (np.sqrt(np.square(convolver.SRC_X)
                      + np.square(convolver.SRC_Y)
                      + np.square(convolver.SRC_Z)) < BRAIN_RADIUS - SRC_R_MAX))

In [ ]:
print(SRC_IDX.sum())

In [ ]:
convolver_interface = ConvolverInterfaceIndexed(convolver,
                                                model_src.csd,
                                                ROMBERG_WEIGHTS,
                                                SRC_IDX)

### Kernel constructor and cross-kernel constructor

The kernel constructor is an object which is a collection of callables (methods) facilitating construction of base function images at electrodes (_PHI_ matrix) and the kernel matrix.  The cross-kernel constructor is a callable which - based on the _PHI_ matrix and boolean mask of the _CSD_ grid - constructs the cross-kernel.

In [ ]:
from _fast_reciprocal_reconstructor import ckESI_kernel_constructor, ckESI_crosskernel_constructor

In [ ]:
kernel_constructor = ckESI_kernel_constructor()

In [ ]:
CSD_IDX = np.ones(convolver.shape('CSD'),
                  dtype=bool)

In [ ]:
kernel_constructor.create_crosskernel = ckESI_crosskernel_constructor(convolver_interface,
                                                                      CSD_IDX)

As all elements of `CSD_IDX` are true, conversion of the reconstructed CSD vector is as simple as its rearrangement to match the _CSD_ grid of `convolver`.  We declare an auxilary `to_3D()` function for that purpose.

In [ ]:
def to_3D(CSD):
    return CSD.reshape(convolver.shape('CSD'))

### Potential At Electrode: analytical solution of the kCSD forward problem

In [ ]:
from _fast_reciprocal_reconstructor import PAE_Analytical

In [ ]:
pae_kcsd = PAE_Analytical(convolver_interface,
                          potential=model_src.potential)

In [ ]:
%%time
PHI_KCSD = kernel_constructor.create_base_images_at_electrodes(electrodes,
                                                               pae_kcsd)

In [ ]:
%%time
KERNEL_KCSD = kernel_constructor.create_kernel(PHI_KCSD)

In [ ]:
%%time
CROSSKERNEL_KCSD = kernel_constructor.create_crosskernel(PHI_KCSD)

It may take few minutes.

In [ ]:
del PHI_KCSD

### Potential At Electrode: kESI corrected analytical solution of the kCSD forward problem

In [ ]:
from _fast_reciprocal_reconstructor import PAE_AnalyticalCorrectedNumerically

In [ ]:
pae_kesi = PAE_AnalyticalCorrectedNumerically(convolver_interface,
                                              potential=model_src.potential)

In [ ]:
%%time
PHI_KESI = kernel_constructor.create_base_images_at_electrodes(electrodes,
                                                               pae_kesi)

It may take even an hour.

In [ ]:
%%time
KERNEL_KESI = kernel_constructor.create_kernel(PHI_KESI)

In [ ]:
%%time
CROSSKERNEL_KESI = kernel_constructor.create_crosskernel(PHI_KESI)

It may take few minutes.

In [ ]:
del PHI_KESI

# Reconstructor

In [ ]:
from kesi._verbose import _CrossKernelReconstructor as Reconstructor
from kesi._engine import _LinearKernelSolver as KernelSolver

In [ ]:
reconstructor_kcsd = Reconstructor(KernelSolver(KERNEL_KCSD),
                                   CROSSKERNEL_KCSD)

In [ ]:
reconstructor_kesi = Reconstructor(KernelSolver(KERNEL_KESI),
                                   CROSSKERNEL_KESI)

# FEM forward modelling

In [ ]:
import configparser
import dolfin
import scipy.interpolate as si

import FEM.fem_common as fc

We define a FEM forward model. It is a callable, which accepts CSD profile as a callable compatible with `scipy.interpolate.RegularGridInterpolator`.

In [ ]:
class ForwardModel(object):
    GROUNDED_PLATE_AT = -0.088

    def __init__(self, mesh, degree, config):
        self.fm = fc.FunctionManager(mesh, degree, 'CG')
        self.config = configparser.ConfigParser()
        self.config.read(config)
        
        self.V = self.fm.function_space
        mesh = self.fm.mesh

        n = self.V.dim()
        d = mesh.geometry().dim()

        self.dof_coords = self.V.tabulate_dof_coordinates()
        self.dof_coords.resize((n, d))
        
        self.csd_f = self.fm.function()
        
        self.subdomains = self.fm.load_subdomains()
        self.dx = dolfin.Measure("dx")(subdomain_data=self.subdomains)

    @property
    def CONDUCTIVITY(self):
        for section in self.config.sections():
            if self._is_conductive_volume(section):
                yield (self.config.getint(section, 'volume'),
                       self.config.getfloat(section, 'conductivity'))

    def _is_conductive_volume(self, section):
        return (self.config.has_option(section, 'volume')
                and self.config.has_option(section, 'conductivity')) 
        
    def __call__(self, csd_interpolator):
        self.csd_f.vector()[:] = csd_interpolator(self.dof_coords)
        
        dirichlet_bc_gt = dolfin.DirichletBC(self.V,
                                     dolfin.Constant(0),
                                     (lambda x, on_boundary:
                                      on_boundary and x[2] <= self.GROUNDED_PLATE_AT))
        test = self.fm.test_function()
        trial = self.fm.trial_function()
        potential = self.fm.function()
        
        
        dx = self.dx
        a = sum(dolfin.Constant(c)
                * dolfin.inner(dolfin.grad(trial),
                               dolfin.grad(test))
                * dx(i)
                for i, c
                in self.CONDUCTIVITY)
        L = self.csd_f * test * dx
        
        b = dolfin.assemble(L)
        A = dolfin.assemble(a)
        dirichlet_bc_gt.apply(A, b)
        
        solver = dolfin.KrylovSolver("cg", "ilu")
        solver.parameters["maximum_iterations"] = 10000
        solver.parameters["absolute_tolerance"] = 1E-8
        solver.solve(A, potential.vector(), b)
        
        return potential

To simulate the differences between actual and assumed head geometries, we use model with thinner CSF layer for forward modelling.

In [ ]:
%%time
fem = ForwardModel(mesh='FEM/meshes/meshes/four_spheres_csf_1_mm_plain/normal.xdmf',
                   degree=1,
                   config='FEM/model_properties/four_spheres_csf_1_mm.ini')

It may take few minutes.

# Reconstruction

## Visualisation

For volumetric data visualisation we define an auxilary functions `crude_plot_data()` and `plot_csd_reconstruction()`.

In [ ]:
import matplotlib.pyplot as plt
import cbf

In [ ]:
def crude_plot_data(DATA,
                    x=None,
                    y=None,
                    z=None,
                    grid=None,
                    dpi=30,
                    cmap=cbf.bwr,
                    title=None,
                    amp=None):
    wx, wy, wz = DATA.shape
    
    if grid is None:
        ix, iy, iz = [w // 2 if a is None else a
                      for a, w in zip([x, y, z],
                                      [wx, wy, wz])]
        x, y, z = ix, iy, iz
        
    else:
        x, y, z = [g.mean() if a is None else a
                   for a, g in zip([x, y, z],
                                   grid)]
        ix, iy, iz = [np.searchsorted(g, a)
                      for a, g in zip([x, y, z],
                                   grid)]

    fig = plt.figure(figsize=((wx + wy) / dpi,
                              (wz + wy) / dpi))
    if title is not None:
        fig.suptitle(title)
    gs = plt.GridSpec(2, 2,
                      figure=fig,
                      width_ratios=[wx, wy],
                      height_ratios=[wz, wy])

    ax_xz = fig.add_subplot(gs[0, 0])
    ax_xz.set_aspect('equal')
    ax_xz.set_ylabel('Z')
    ax_xz.set_xlabel('X')

    ax_yx = fig.add_subplot(gs[1, 1])
    ax_yx.set_aspect('equal')
    ax_yx.set_ylabel('X')
    ax_yx.set_xlabel('Y')

    ax_yz = fig.add_subplot(gs[0, 1],
                            sharey=ax_xz,
                            sharex=ax_yx)
    ax_yz.set_aspect('equal')

    cax = fig.add_subplot(gs[1, 0])
    cax.set_visible(False)

    if amp is None:
        amp = abs(DATA).max()

    if grid is None:
        ax_xz.imshow(DATA[:, iy, :].T,
                     vmin=-amp,
                     vmax=amp,
                     cmap=cmap,
                     origin='lower')
        ax_yx.imshow(DATA[:, :, iz],
                     vmin=-amp,
                     vmax=amp,
                     cmap=cmap,
                     origin='lower')
        im = ax_yz.imshow(DATA[ix, :, :].T,
                          vmin=-amp,
                          vmax=amp,
                          cmap=cmap,
                          origin='lower')
    else:
        ax_xz.imshow(DATA[:, iy, :].T,
                     vmin=-amp,
                     vmax=amp,
                     cmap=cmap,
                     origin='lower',
                     extent=(grid[0].min(), grid[0].max(),
                             grid[2].min(), grid[2].max()))
        ax_yx.imshow(DATA[:, :, iz],
                     vmin=-amp,
                     vmax=amp,
                     cmap=cmap,
                     origin='lower',
                     extent=(grid[1].min(), grid[1].max(),
                             grid[0].min(), grid[0].max()))
        im = ax_yz.imshow(DATA[ix, :, :].T,
                          vmin=-amp,
                          vmax=amp,
                          cmap=cmap,
                          origin='lower',
                          extent=(grid[1].min(), grid[1].max(),
                                  grid[2].min(), grid[2].max()))
        
    ax_xz.axvline(x, ls=':', color=cbf.BLACK)
    ax_xz.axhline(z, ls=':', color=cbf.BLACK)

    ax_yx.axvline(y, ls=':', color=cbf.BLACK)
    ax_yx.axhline(x, ls=':', color=cbf.BLACK)

    ax_yz.axvline(y, ls=':', color=cbf.BLACK)
    ax_yz.axhline(z, ls=':', color=cbf.BLACK)
    fig.colorbar(im, ax=cax)

    return (fig, ((ax_xz, ax_yz),
                  (cax, ax_yx)))

In [ ]:
csd_grid = [_x.flatten() for _x in convolver.CSD_MESH]

In [ ]:
def add_spheres(plot, x, y, z,
                sphere_radii=[0.079, 0.082, 0.086, 0.090]):
    def plot_circle(ax, r):
        ax.add_artist(plt.Circle((0, 0), r,
                                facecolor='none',
                                edgecolor=cbf.BLACK,
                                linestyle=':'))

    (fig, ((ax_xz, ax_yz),
           (cax, ax_yx))) = plot
    for r2 in np.square(sphere_radii):
        plot_circle(ax_xz, np.sqrt(r2 - np.square(y)))
        plot_circle(ax_yz, np.sqrt(r2 - np.square(x)))
        plot_circle(ax_yx, np.sqrt(r2 - np.square(z)))

def plot_csd_reconstruction(CSD, x, y, z, title=''):
    ERROR = CSD - GT_CSD
    error_L2 = np.sqrt(np.square(ERROR).sum() / np.square(GT_CSD).sum())
    amp = max(abs(CSD).max(),
              abs(GT_CSD).max(),
              abs(ERROR).max())
    
    for plot in [crude_plot_data(GT_CSD, title='GT CSD',
                                 grid=csd_grid, x=x, y=y, z=z, dpi=35,
                                 amp=amp),
                 crude_plot_data(CSD, title=f'{title} reconstruction',
                                 grid=csd_grid, x=x, y=y, z=z, dpi=35,
                                 amp=amp),
                 crude_plot_data(ERROR, title=f'{title} error (GT normalized L2 norm: {error_L2:.2g})',
                                 grid=csd_grid, x=x, y=y, z=z, dpi=35,
                                 amp=amp)]:
        add_spheres(plot, x, y, z)

## Ground truth CSD and its potential at the electrodes

Derive GT CSD as an eigensource of analytical kCSD.  We reverse order of the returned eigenvalues (and eigenvectors) for the sake of the highest-first convention.

In [ ]:
%%time
EIGENVALUES_KCSD, EIGENVECTORS_KCSD = np.linalg.eigh(KERNEL_KCSD)
EIGENVALUES_KCSD, EIGENVECTORS_KCSD = EIGENVALUES_KCSD[::-1], EIGENVECTORS_KCSD[:, ::-1]

GT_CSD = to_3D(reconstructor_kcsd(EIGENVECTORS_KCSD[:, 2]))

In [ ]:
add_spheres(crude_plot_data(GT_CSD,
                            title='GT CSD',
                            grid=csd_grid,
                            x=-0.05,
                            y=0.015,
                            z=-0.02,
                            dpi=35),
            x=-0.05,
            y=0.015,
            z=-0.02,
            sphere_radii=[0.079, 0.080, 0.085, 0.090])

Simulate the potential generated by the ground truth CSD profile.

In [ ]:
%%time
_csd = si.RegularGridInterpolator(csd_grid,
                                  GT_CSD,
                                  bounds_error=False,
                                  fill_value=0)
potential = fem(_csd)

We probe the potential in the whole region of interest to visualise it (it may take several minutes).

In [ ]:
%%time
with np.nditer([convolver.CSD_X[::2, :, :],
                convolver.CSD_Y[:, ::2, :],
                convolver.CSD_Z[:, :, ::2],
                None]) as it:
    for _x, _y, _z, _res in it:
        try:
            _res[...] = potential(_x, _y, _z)
        except RuntimeError:
            _res[...] = np.nan

    V_ROI = np.ma.masked_invalid(it.operands[3])

In [ ]:
add_spheres(crude_plot_data(V_ROI,
                            cmap=cbf.PRGn,
                            grid=[_x[::2] for _x in csd_grid],
                            x=-0.05,
                            y=0.015,
                            z=-0.02,
                            dpi=17,
                            title='POTENTIAL'),
            x=-0.05,
            y=0.015,
            z=-0.02,
            sphere_radii=[0.079, 0.080, 0.085, 0.090])

Probe the potential at electrodes.

In [ ]:
GT_V = np.array([potential(_e.x, _e.y, _e.z) for _e in electrodes])

To make the case more realistic we introduce 1% noise.

In [ ]:
np.random.seed(42)
NOISE_V = np.random.normal(loc=GT_V,
                           scale=0.01 * GT_V.std())

In [ ]:
CSD_KCSD = to_3D(reconstructor_kcsd(NOISE_V))

In [ ]:
CSD_KESI = to_3D(reconstructor_kesi(NOISE_V))

In [ ]:
plot_csd_reconstruction(CSD_KCSD,
                        x=-0.05,
                        y=0.015,
                        z=-0.02,
                        title='kCSD')

In [ ]:
plot_csd_reconstruction(CSD_KESI,
                        x=-0.05,
                        y=0.015,
                        z=-0.02,
                        title='kESI')

As you see, while the kCSD reconstruction is mostly error, the kESI reconstruction somehow resembles the ground truth.

## Regularization

### Cross-validation

To counter the noise we select a regularization parameter which minimizes cross-validation error.  The span of parameters from which it is being selected should cover eigenvalues of the kernel.

In [ ]:
EIGENVALUES_KESI = np.linalg.eigvalsh(KERNEL_KESI)[::-1]

We reversed order of the returned eigenvalues for the sake of the highest-first convention.

In [ ]:
plt.figure(figsize=(18, 12))
plt.plot(EIGENVALUES_KCSD, label='kCSD', ls='-')
plt.plot(EIGENVALUES_KESI, label='kESI', ls=':')
plt.yscale('log')
plt.legend(loc='best')

In [ ]:
REGULARIZATION_PARAMETERS = np.logspace(-2, 15, 17 * 10 + 1)

In [ ]:
from _common_new import cv

In [ ]:
%%time
CV_ERRORS_KCSD = cv(reconstructor_kcsd, NOISE_V, REGULARIZATION_PARAMETERS)

It may take several minutes if other tasks are running.

In [ ]:
%%time
CV_ERRORS_KESI = cv(reconstructor_kesi, NOISE_V, REGULARIZATION_PARAMETERS)

It may take several minutes if other tasks are running.

In [ ]:
regularization_parameter_kesi = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KESI)]
regularization_parameter_kcsd = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KCSD)]

In [ ]:
_kcsd = plt.plot(REGULARIZATION_PARAMETERS,
                 CV_ERRORS_KCSD,
                 label='kCSD')
_kesi = plt.plot(REGULARIZATION_PARAMETERS,
                 CV_ERRORS_KESI,
                 label='kESI')
plt.axvline(regularization_parameter_kcsd,
            ls=':',
            color=_kcsd[0].get_color())
plt.axvline(regularization_parameter_kesi,
            ls=':',
            color=_kesi[0].get_color())
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')
plt.legend(loc='best')

In [ ]:
CSD_KCSD_CV = to_3D(reconstructor_kcsd(NOISE_V, regularization_parameter_kcsd))

In [ ]:
CSD_KESI_CV = to_3D(reconstructor_kesi(NOISE_V, regularization_parameter_kesi))

In [ ]:
plot_csd_reconstruction(CSD_KCSD_CV,
                        x=-0.05,
                        y=0.015,
                        z=-0.02,
                        title='kCSD (regularized)')

The reconstruction still is mostly error.

In [ ]:
plot_csd_reconstruction(CSD_KESI_CV,
                        x=-0.05,
                        y=0.015,
                        z=-0.02,
                        title='kESI (regularized)')

The kESI reconstruction definitely benefitted from regularization.